In [1]:
!pip install --upgrade polars
!pip install nfl_data_py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 16.8 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.17.3
    Uninstalling polars-0.17.3:
      Successfully uninstalled polars-0.17.3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.0 MB/s eta 0:00:00
  Created wheel for nfl_data_py: filename=nfl_data_py-0.3.1-py3-none-any.whl size=13187 sha256=f7f130f172ddc78d1c7dcc189deaac90b10bf49839a146409214380ce30c9662
  Stored in directory: /root/.cache/pip/wheels/ca/2e/d2/4def312ac7c070e1c0002e91ecc6923481a86c3ebd5b2c5413
Successfully built nfl_data_py


In [2]:
import os
import pandas as pd
import polars as pl
import nfl_data_py as nfl


from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
os.chdir("/content/gdrive/MyDrive/NFL_Challenge/QB-GPT/data_preprocessing/0_raw")

In [4]:
os.listdir()

['Contact Detection',
 'Punt Prediction',
 'Analytics',
 'Impact Detection',
 'data bowl 2021',
 'data bowl 2023',
 'data bowl 2022',
 'data bowl 2020',
 'asonty',
 'Highlights_NGS_2019',
 'Highlights_NGS_Prime',
 'data bowl 2024',
 'final_df_2024.parquet']

In [5]:
to_get = [
 'data bowl 2021',
 'data bowl 2022',
 'data bowl 2023',
 'data bowl 2024',
 'asonty',
 'Highlights_NGS_2019']

In [6]:
def read(p):
  data = pl.read_parquet(p)
  to_insert = p.replace("/processed_df.parquet", "")
  data = (data.
          with_columns(pl.col("nflId").cast(pl.Int64)).
          with_columns(pl.lit(to_insert).alias("data_source")))
  return data

In [7]:
years_to_get = [2018, 2019, 2020, 2021, 2022, 2023]
season_data = pl.from_pandas(nfl.import_pbp_data(years_to_get))
rosters = pl.from_pandas(nfl.import_seasonal_rosters(years_to_get))

2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
Downcasting floats.


In [8]:
dataset = pl.concat([read(p+"/processed_df.parquet").unique() for p in to_get])

In [9]:
(dataset.
 head())

PlayType,gameId,playId,team,OffDef,nflId,position,frameId,x,y,data_source
str,i64,i64,str,str,i64,str,i64,f64,f64,str
"""Passing""",2018091612,260,"""NE""","""Defense""",41269,"""OLB""",1,60.93,24.55,"""data bowl 2021…"
"""Passing""",2018091612,260,"""NE""","""Defense""",41269,"""OLB""",7,60.65,24.61,"""data bowl 2021…"
"""Passing""",2018091612,260,"""NE""","""Defense""",41269,"""OLB""",9,60.58,24.6,"""data bowl 2021…"
"""Passing""",2018091612,260,"""NE""","""Defense""",41269,"""OLB""",11,60.53,24.57,"""data bowl 2021…"
"""Passing""",2018091612,260,"""NE""","""Defense""",41269,"""OLB""",15,60.4,24.42,"""data bowl 2021…"


In [10]:
dataset = (dataset.
           with_columns(pl.when(pl.col("team") == "OAK").
                        then(pl.lit("LV")).
                        otherwise(pl.col("team"))).
           group_by('data_source', 'PlayType', 'gameId', 'playId', 'team', 'OffDef', 'nflId', 'position').
           agg([pl.col("frameId"), pl.col("x"), pl.col("y")]).
           filter(pl.col("nflId").is_not_null()).
           join(
              season_data.
              select("old_game_id", "play_id", "posteam").
              unique().
              with_columns([
                  pl.col("old_game_id").cast(pl.Int64).alias("gameId"),
                  pl.col("play_id").cast(pl.Int64).alias("playId")
              ]).
              drop("old_game_id", "play_id"),
              on = ["gameId", "playId"],
              how = "left"
           ).
           with_columns(pl.when(pl.col("posteam") == pl.col("team")).
                        then(pl.lit("Offense")).
                        otherwise(pl.lit("Defense")).
                        alias("new_team")).
           drop("OffDef", "posteam").
           rename({"new_team" : "OffDef"}).
           join(season_data.
                select("old_game_id", "play_id", "yardline_100", "qtr", "down", "play_type", "season_type", "season").
                with_columns([
                    pl.col("old_game_id").cast(pl.Int64).alias("gameId"),
                    pl.col("play_id").cast(pl.Int64).alias("playId")
                    ]).
                drop("old_game_id", "play_id"),
                on = ["gameId", "playId"],
                how = "left").
           drop("PlayType").
           rename({"play_type" : "PlayType"}).
           with_columns(pl.col("frameId").list.lengths().alias("NB_Frames")).
           select("data_source", "season_type", "season", "gameId", "qtr", "down", "yardline_100", "playId", "PlayType", "team", "OffDef", "nflId", "position", "frameId", "x", "y").
           unique(subset = ["gameId", "playId", "nflId"]))


filter_NB_players = (dataset.
                     select("gameId", "playId", "OffDef", "nflId").
                     unique().
                     group_by("gameId", "playId", "OffDef").
                     count().
                     rename({"count" : "Nb_players"}))

dataset = (dataset.
           join(filter_NB_players,
                on = ["gameId", "playId", "OffDef"],
                how = "left").
           filter(pl.col("Nb_players") > 1).
           drop("Nb_players"))

<ipython-input-10-28fc45d07567>:37: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len`.
  with_columns(pl.col("frameId").list.lengths().alias("NB_Frames")).


In [11]:
import gc
_ = gc.collect()

In [12]:
uniques = (dataset.
           select("gameId", "playId", "nflId").
           unique())

dataset = (uniques.
           join(dataset,
                on = ["gameId", "playId", "nflId"]))

## V1

In [14]:
specific_to_add = (dataset.
                   select("data_source", "season", "gameId", "playId", "OffDef", "nflId").
                   group_by("data_source", "season", "gameId", "playId", "OffDef").
                   agg(pl.col("nflId").cast(pl.Utf8).alias("players_scraped")).
                   join(
                      season_data.
                      with_columns(
                          [
                              pl.col("old_game_id").cast(pl.Int64).alias("gameId"),
                              pl.col("play_id").cast(pl.Int64).alias("playId")
                              ]).
                      select("season", "gameId", "playId", "offense_players", "defense_players").
                      melt(id_vars = ["season", "gameId", "playId"],
                           value_vars = ["offense_players", "defense_players"],
                           variable_name = "OffDef",
                           value_name = "player_id").
                      with_columns(pl.when(pl.col("OffDef") == "offense_players").
                                   then(pl.lit("Offense")).
                                   otherwise(pl.lit("Defense")).
                                   alias("OffDef")).
                      with_columns(pl.col("player_id").str.split(";")).
                      explode("player_id").
                      join(
                          rosters.
                          select("player_id", "gsis_it_id").
                          unique().
                          rename({"gsis_it_id" : "nflId"}),
                          on = "player_id",
                          how = "left"
                      ).
                      drop("player_id").
                      group_by("season", "gameId", "playId", "OffDef").
                      agg(pl.col("nflId").alias("players_present")),
                      on = ["season", "gameId", "playId", "OffDef"],
                      how = "left").
                   with_columns(pl.col("players_present").list.set_difference("players_scraped").alias("difference")).
                   select("data_source", "season", "gameId", "playId", "OffDef", "difference").
                   explode("difference").
                   filter(pl.col("difference") != "").
                   rename({"difference" : "nflId"}).
                   with_columns(pl.col("nflId").cast(pl.Int64)).
                   join(
                      rosters.
                      select("season", "gsis_it_id", "position").
                      with_columns([
                          pl.col("season").cast(pl.Int64).alias("season"),
                          pl.col("gsis_it_id").cast(pl.Int64).alias("nflId")
                          ]).
                      drop("gsis_it_id").
                      unique(subset=["season", "nflId"]),
                      on = ["season", "nflId"],
                      how = "left").
                   fill_null("UNKNOWN").
                   join(
                      dataset.
                      select("season", "gameId", "playId", "season_type", "team", "OffDef", "qtr", "down", "yardline_100", "PlayType").
                      unique(),
                      on = ["season", "gameId", "playId", "OffDef"],
                      how = "left").
                   select('data_source', 'season_type', 'season', 'gameId', 'qtr', 'down', 'yardline_100', 'playId', 'PlayType', 'team', 'OffDef', 'nflId', 'position').
                   with_columns([
                      pl.lit(-1).cast(pl.Int64).alias("frameId"),
                      pl.lit(-1).cast(pl.Float64).alias("x"),
                      pl.lit(-1).cast(pl.Float64).alias("y")
                   ]).
                   group_by('data_source', 'season_type', 'season', 'gameId', 'qtr', 'down', 'yardline_100', 'playId', 'PlayType', 'team', 'OffDef', 'nflId', 'position').
                   agg([pl.col("frameId"),
                        pl.col("x"),
                        pl.col("y")]).
                   with_columns(pl.lit(-1).alias("Length")))

specific_to_keep = (dataset.
                    select("data_source", "season", "gameId", "playId", "nflId").
                    group_by("data_source", "season", "gameId", "playId").
                    agg(pl.col("nflId").cast(pl.Utf8).alias("players_scraped")).
                    join(
                        season_data.
                        with_columns(
                            [
                                pl.col("old_game_id").cast(pl.Int64).alias("gameId"),
                                pl.col("play_id").cast(pl.Int64).alias("playId")
                                ]).
                        select("gameId", "playId", "players_on_play").
                        with_columns(pl.col("players_on_play").str.split(";").alias("players_present")),
                        on = ["gameId", "playId"],
                        how = "left").
                    drop("players_on_play").
                    with_columns(pl.col("players_present").list.set_intersection("players_scraped").alias("kept")).
                    select("data_source", "season", "gameId", "playId", "kept").
                    explode("kept").
                    filter(pl.col("kept") != "").
                    rename({"kept" : "nflId"}).
                    with_columns(pl.col("nflId").cast(pl.Int64)).
                    join(
                        dataset,
                        on = ["season", "gameId", "playId", "nflId"],
                        how = "left").
                    select('data_source', 'season_type', 'season', 'gameId', 'qtr', 'down', 'yardline_100', 'playId', 'PlayType', 'team', 'OffDef', 'nflId', 'position', 'frameId', 'x', 'y').
                    with_columns(pl.col("frameId").list.lengths().cast(pl.Int32).alias("Length")))

<ipython-input-14-fc3e7a084d90>:99: DeprecationWarning: `lengths` is deprecated. It has been renamed to `len`.
  with_columns(pl.col("frameId").list.lengths().cast(pl.Int32).alias("Length")))


In [15]:
dataset = (pl.concat([specific_to_keep, specific_to_add]).
           filter(pl.col("Length") != -1))

In [ ]:
(dataset.
 select("gameId", "playId", "team", "OffDef", "nflId").
 unique().
 shape)

(1170107, 5)

In [16]:
(dataset.head())

data_source,season_type,season,gameId,qtr,down,yardline_100,playId,PlayType,team,OffDef,nflId,position,frameId,x,y,Length
str,str,i64,i64,f32,f32,f32,i64,str,str,str,i64,str,list[i64],list[f64],list[f64],i32
"""data bowl 2021…","""REG""",2018,2018091612,1.0,2.0,52.0,260,"""pass""","""JAX""","""Offense""",44923,"""WR""","[3, 5, … 49]","[56.37, 56.36, … 64.63]","[34.8, 34.8, … 35.3]",27
"""data bowl 2021…","""REG""",2018,2018091612,1.0,2.0,52.0,260,"""pass""","""NE""","""Defense""",38540,"""CB""","[1, 3, … 53]","[58.87, 58.89, … 73.29]","[9.94, 9.92, … -0.93]",27
"""data bowl 2021…","""REG""",2018,2018091612,1.0,2.0,52.0,260,"""pass""","""JAX""","""Offense""",41267,"""TE""","[1, 7, … 47]","[57.0, 57.03, … 62.08]","[28.78, 28.79, … 23.41]",27
"""data bowl 2021…","""REG""",2018,2018091612,1.0,2.0,52.0,260,"""pass""","""NE""","""Defense""",38555,"""MLB""","[5, 7, … 53]","[58.71, 58.71, … 53.98]","[21.3, 21.29, … 23.02]",27
"""data bowl 2021…","""REG""",2018,2018091612,1.0,2.0,52.0,260,"""pass""","""NE""","""Defense""",35466,"""FS""","[1, 3, … 51]","[73.86, 73.89, … 79.74]","[24.98, 24.97, … 14.4]",27


In [17]:
dataset.write_parquet("final_df_2024.parquet")